<a href="https://colab.research.google.com/github/vchraf/GeniusLyricsScraper/blob/main/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import requests
import urllib.request
import urllib.parse
import json
import re
from bs4 import BeautifulSoup

In [2]:
BASE = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = "-q1tRGBZMEOk6JewZCC_KWZBxyFSg9nccGlX11Cb3MxpGpzWG4FBSJIXCJS33D3x"

In [3]:
id    = 983488
path  = f"artists/{id}/songs/"
query  = f"{BASE}/{path}"
token = f"Bearer {CLIENT_ACCESS_TOKEN}"
headers = {"Authorization": token}
params = {'page': 2}
response = requests.get(url=query, params=params, headers=headers)

In [4]:
print(query)

https://api.genius.com/artists/983488/songs/


In [6]:
r = response.json()

In [43]:
songsList = [song["id"] for song in r['response']['songs'] if song["primary_artist"]["id"] == id]
songID = songsList[1]

In [44]:
url = f"songs/{songID}"
query  = f"{BASE}/{url}"
token = f"Bearer {CLIENT_ACCESS_TOKEN}"
headers = {"Authorization": token}
params = {'page': 2}
response = requests.get(url=query, params=params, headers=headers)
data = response.json()

In [62]:
URL = "http://genius.com" + data['response']['song']['path']
print(URL)
page = requests.get(URL)
html = BeautifulSoup(page.text, "html.parser")
js = html.find_all("script")[16]
txt = str(js)[str(js).find('\\"html\\":\\"<p><b>')+11 : str(js).find('\\"children\\":[{\\"children\\') - 2]
txt = re.sub('<.*?>', '',txt)
txtli = txt.split('\\n')
[t.replace("\\","") for t in txtli if t != '\\' ]

http://genius.com/Hamza15-3-jpg-lyrics


['[Paroles de Hamza 15-3 "JPG]',
 '[Couplet 1]',
 "4k fl'vision, microphone matser Ki Das efx",
 "Spielberg science fiction, katb diaski bl'After effect",
 'M\'signé m3a rap contrat, jayb "TILT" f\'kol track',
 "Zayd 3lik neghza f l'contraste",
 'Wakha tkoun kattir, lyouma gha takol trab',
 'Haz Meknes fo9 lktaf ki Atlas',
 'Ma kant3na 7ta bchi blan matrach',
 'Machi 7ta nmout 3ad ygolo kan rajl',
 'Mehrat, mnin ma dzt mkhalli la trace',
 "Rappit ghi 7it rap kay'sstahel",
 "W ghi b'rrit 9rit ch7al mn jahel",
 "Kanji face w ga3 ma kan'jamel",
 "Mjhed 3lihom w fihom kan'jahed",
 "Ki lakers d'Kobe w Shaq",
 '9alb tbla 3la ga3 li ba9i chak',
 "Ma kansaynch l'mssida tched",
 '9atl koulchi, 7ta li kan ghi chahd',
 'Ghost rider ki Nicola Cage',
 'Solo w la jit b7al la daz cortège',
 'Kan7et ghi ssem Ki Scorsesse',
 "Texte sawi 3liha m'escorteh",
 'Golden boy, van persie (yeah)',
 "3ndi ki l'Mnouni ki Bercy (same)",
 'No drama, no mercy',
 "Imbatable b7al Wojtek f'RC",
 "Kanrsm tssawer gha b's

In [60]:
str(js).find('\\"html\\":\\"<p><b>')

430

In [59]:
str(js)[430:]

'\\"html\\":\\"<p><b>[Paroles de Hamza 15-3 \\\\\\"JPG]<\\/b><br>\\\\n<br>\\\\n[Couplet 1]<br>\\\\n<a href=\\\\\\"/21084436/Hamza15-3-jpg/4k-flvision-microphone-matser-ki-das-efx\\\\\\" data-id=\\\\\\"21084436\\\\\\" class=\\\\\\"has_pinned_annotation has_verified_annotation\\\\\\" data-editorial-state=\\\\\\"needs_exegesis\\\\\\" data-classification=\\\\\\"verified\\\\\\">4k fl\\\'vision, microphone matser Ki Das efx<\\/a><br>\\\\nSpielberg science fiction, katb diaski bl\\\'After effect<br>\\\\nM\\\'signé m3a rap contrat, jayb \\\\\\"TILT\\\\\\" f\\\'kol track<br>\\\\nZayd 3lik neghza f l\\\'contraste<br>\\\\nWakha tkoun kattir, lyouma gha takol trab<br>\\\\n<a href=\\\\\\"/21084476/Hamza15-3-jpg/Haz-meknes-fo9-lktaf-ki-atlas\\\\\\" data-id=\\\\\\"21084476\\\\\\" class=\\\\\\"has_pinned_annotation has_verified_annotation\\\\\\" data-editorial-state=\\\\\\"needs_exegesis\\\\\\" data-classification=\\\\\\"verified\\\\\\">Haz Meknes fo9 lktaf ki Atlas<\\/a><br>\\\\nMa kant3na 7ta bchi b